In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import glob
import os

In [ ]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016]

for year in years:
    base = 'http://profiles.doe.mass.edu/state_report/mobilityrates.aspx?&export_excel=yes&ctl00$ContentPlaceHolder1$cohortYear={}&ctl00$ContentPlaceHolder1$reportType=SCHOOL&ctl00$ContentPlaceHolder1$rateType=4-Year:REG&ctl00$ContentPlaceHolder1$studentGroup=ALL'
    base = base.format(year)
    print(base)
    filename = "student_"+"mobility_"+str(year)+".xls"
    with open(filename, 'w') as output:
        output.write(requests.get(base).text)

In [ ]:
## Create function to clean up DFs

In [9]:
def clean_file(file):
    df = pd.read_html(file)
    df = pd.DataFrame(df[1])
    header = df.iloc[0]
    df.columns = header
    df = df[1:]
    df.set_index('OrgCode')
    file_name = os.path.splitext(file)[0]
    df["Source"] = file_name
    df['Year'] = df["Source"].str[-4:]
    return df

clean_file("student_mobility_2010.xls")

,Org Name,OrgCode,Churn Enrollment,% Churn,% Intake,Stability Enrollment,% Stability,Source,Year
1,Abby Kelley Foster Charter Public (District) -...,04450105,1465,7.2,4.4,1435,94.7,student_mobility_2010,2010
2,Abington - Abington ECC,00010003,562,11.0,8.2,536,93.3,student_mobility_2010,2010
3,Abington - Abington High,00010505,593,10.8,5.9,577,91.7,student_mobility_2010,2010
4,Abington - Center,00010005,250,10.8,7.2,240,92.9,student_mobility_2010,2010
5,Abington - Frolio Middle School,00010405,415,6.8,4.3,408,94.9,student_mobility_2010,2010
6,Abington - Woodsdale,00010015,486,5.6,3.9,481,95.4,student_mobility_2010,2010
7,Academy Of the Pacific Rim Charter Public (Dis...,04120530,487,4.7,1.9,485,95.7,student_mobility_2010,2010
8,Acton - Douglas,00020020,506,5.1,1.8,499,96.2,student_mobility_2010,2010
9,Acton - Gates,00020025,502,5.2,2.8,491,97.0,student_mobility_2010,2010
10,Acton - Luther Conant,00020030,508,2.6,1.8,500,99.0,student_mobility_2010,2010


In [ ]:
# Create master CSV

In [10]:
all_files = glob.glob("student_mobility_201[1-6].xls")
df_list = []

for file in all_files:
    file = clean_file(file)
    df_list.append(file)
    
df = pd.concat(df_list)

In [12]:
df = df.rename(columns={'OrgCode': 'Org Code'})
df

,Org Name,Org Code,Churn Enrollment,% Churn,% Intake,Stability Enrollment,% Stability,Source,Year
1,Abby Kelley Foster Charter Public (District) -...,04450105,1444,4.1,2.8,1427,97.1,student_mobility_2011,2011
2,Abington - Abington ECC,00010003,687,13.5,11.1,635,93.5,student_mobility_2011,2011
3,Abington - Abington High,00010505,538,11.3,4.8,523,91.2,student_mobility_2011,2011
4,Abington - Frolio Middle School,00010405,378,5.6,2.7,373,95.7,student_mobility_2011,2011
5,Abington - Woodsdale,00010015,525,7.1,3.8,511,95.5,student_mobility_2011,2011
6,Academy Of the Pacific Rim Charter Public (Dis...,04120530,487,4.5,2.5,485,95.9,student_mobility_2011,2011
7,Acton - Douglas,00020020,493,1.6,1.0,488,99.4,student_mobility_2011,2011
8,Acton - Gates,00020025,495,3.0,1.0,490,98.0,student_mobility_2011,2011
9,Acton - Luther Conant,00020030,497,3.0,1.8,488,98.8,student_mobility_2011,2011
10,Acton - McCarthy-Towne,00020015,502,5.4,3.4,486,97.7,student_mobility_2011,2011


In [ ]:
# Test out mask

In [13]:
mask = df["Org Code"] == "04450105"
df_abbyk = df[mask]

df_abbyk

,Org Name,Org Code,Churn Enrollment,% Churn,% Intake,Stability Enrollment,% Stability,Source,Year
1,Abby Kelley Foster Charter Public (District) -...,04450105,1444,4.1,2.8,1427,97.1,student_mobility_2011,2011
1,Abby Kelley Foster Charter Public (District) -...,04450105,1442,5.0,3.7,1426,96.1,student_mobility_2012,2012
1,Abby Kelley Foster Charter Public (District) -...,04450105,1436,4.7,3.7,1428,95.8,student_mobility_2013,2013
1,Abby Kelley Foster Charter Public (District) -...,04450105,1451,7.4,5.6,1438,93.5,student_mobility_2014,2014
1,Abby Kelley Foster Charter Public (District) -...,04450105,1443,4.5,3.2,1429,96.4,student_mobility_2015,2015
1,Abby Kelley Foster Charter Public (District) -...,04450105,1436,3.2,2.0,1429,97.3,student_mobility_2016,2016


In [ ]:
# Set OrgCode as index and write master file to CSV

In [15]:
df = df.set_index('Org Code')
df.to_csv("student_mobility_all.csv")